In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
sys.path.append('../')
%store -r dm
dm = dm

no stored variable or alias dm


NameError: name 'dm' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

X_A_train = dm.data_A.iloc[:, 2:-1]
y_A_train = dm.data_A.iloc[:,0]
X_B_train = dm.data_B.iloc[:, 2:-1]
y_B_train = dm.data_B.iloc[:,0]
X_C_train = dm.data_C.iloc[:, 2:-1]
y_C_train = dm.data_C.iloc[:,0]


X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(X_A_train, y_A_train, shuffle=False, train_size=0.9)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(X_B_train, y_B_train, shuffle=False, train_size=0.9)
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(X_C_train, y_C_train, shuffle=False, train_size=0.9)

X_A_submission = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_submission = dm.X_test_estimated_b[dm.X_test_estimated_b.columns.intersection(X_B_train.columns)]
X_C_submission = dm.X_test_estimated_c[dm.X_test_estimated_c.columns.intersection(X_C_train.columns)]

print(X_train_A.shape, y_train_A.shape, X_test_A.shape, y_test_A.shape)
print(X_train_B.shape, y_train_B.shape, X_test_B.shape, y_test_B.shape)
print(X_train_C.shape, y_train_C.shape, X_test_C.shape, y_test_C.shape)



In [ ]:
from tpot import TPOTRegressor

In [ ]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train_A, y_train_A)
print(tpot.score(X_test_A, y_test_A))
tpot.export('tpot_pipeline.py')

## Best Model:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: -218261.99683292146
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    RandomForestRegressor(bootstrap=True, max_features=0.45, min_samples_leaf=1, min_samples_split=19, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


# AutoKeras

In [1]:
%pip install autokeras

  Using cached autokeras-1.1.0-py3-none-any.whl (148 kB)
  Using cached keras_nlp-0.6.2-py3-none-any.whl.metadata (7.2 kB)
  Using cached keras_core-0.1.7-py3-none-any.whl.metadata (4.3 kB)
  Using cached regex-2023.10.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached rich-13.6.0-py3-none-any.whl.metadata (18 kB)
  Using cached dm_tree-0.1.8-cp311-cp311-macosx_11_0_arm64.whl (110 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached keras_nlp-0.6.2-py3-none-any.whl (590 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
Using cached keras_core-0.1.7-py3-none-any.whl (950 kB)
Using cached regex-2023.10.3-cp311-cp311-macosx_11_0_arm64.whl (291 kB)
Using cached rich-13.6.0-py3-none-any.whl (239 kB)
Using cached markdown_it_py-3.0.0-py3

In [2]:
import autokeras as ak

Using TensorFlow backend


In [5]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=100
)  # It tries 3 different models.
# Feed the structured data regressor with training data.
train_file_path = "../csv_files/All_raw_non_NaN.csv"

reg.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    "pv_measurement",
    epochs=10,
)


# Predict with the best model.
#predicted_y = reg.predict(test_file_path)
# Evaluate the best model with testing data.
#print(reg.evaluate(test_file_path, "pv_measurement"))

Trial 49 Complete [00h 01m 05s]
val_loss: 482057.5625

Best val_loss So Far: 419263.71875
Total elapsed time: 14h 44m 16s


Epoch 1/10
2907/2907 [==============================] - 6s 2ms/step - loss: 657122.8125 - mean_squared_error: 657122.8125
Epoch 2/10
2907/2907 [==============================] - 5s 2ms/step - loss: 575622.8750 - mean_squared_error: 575622.8750
Epoch 3/10
2907/2907 [==============================] - 5s 2ms/step - loss: 519078.7500 - mean_squared_error: 519078.7500
Epoch 4/10
2907/2907 [==============================] - 5s 2ms/step - loss: 501818.3125 - mean_squared_error: 501818.3125
Epoch 5/10
2907/2907 [==============================] - 5s 2ms/step - loss: 494477.3438 - mean_squared_error: 494477.3438
Epoch 6/10
2907/2907 [==============================] - 5s 2ms/step - loss: 490229.1562 - mean_squared_error: 490229.1562
Epoch 7/10
2907/2907 [==============================] - 6s 2ms/step - loss: 487661.1562 - mean_squared_error: 487661.1562
Epoch 8/10
2907/2907 [==============================] - 6s 2ms/step - loss: 483531.5312 - mean_squared_error: 483531.5312
Epoch 9/10
2907/2907 [==

INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets
